In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        pass

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
import os
import cv2
import numpy as np
from glob import glob
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical, Sequence
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout, Input
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import albumentations as A
import tensorflow.keras.backend as K
import tensorflow as tf

# Custom Focal Loss Function
def categorical_focal_loss(alpha=0.25, gamma=2.0):
    def loss(y_true, y_pred):
        y_pred = K.clip(y_pred, K.epsilon(), 1. - K.epsilon())
        cross_entropy = -y_true * K.log(y_pred)
        weight = alpha * K.pow(1 - y_pred, gamma)
        loss = weight * cross_entropy
        return K.sum(loss, axis=1)
    return loss

# Configuration
CONFIG = {
    "BATCH_SIZE": 128,
    "NUM_CLASSES": 27,
    "LR": 0.0001,
    "EPOCHS": 50
}

# Dataset Paths
TRAIN_PATH = "/kaggle/input/train-alphabet1"
VALIDATION_PATH = "/kaggle/input/test-dataset1"
SELECTED_CLASSES = [
    'A', 'B', 'Blank' , 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M',
    'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z'
]
class_to_idx = {cls: idx for idx, cls in enumerate(SELECTED_CLASSES)}

# Albumentations Transforms
train_transform = A.Compose([
    A.OneOf([
        A.RandomBrightnessContrast(0.2, 0.2, p=0.5),
        A.CLAHE(clip_limit=2.0, p=0.5)
    ], p=0.4),
    A.HueSaturationValue(5, 10, 10, p=0.3),
    A.ImageCompression(quality_lower=85, quality_upper=100, p=0.2),
    A.Equalize(p=0.1),
    A.ShiftScaleRotate(0.02, 0.02, 9, border_mode=0, p=0.5),
    A.MotionBlur(blur_limit=(3, 5), p=0.1),
])

val_transform = A.Compose([])  # No augmentation for validation

MAX_IMAGES_PER_CLASS = 750

# Load Training Data
train_image_paths, train_labels = [], []
for class_name in SELECTED_CLASSES:
    folder = os.path.join(TRAIN_PATH, class_name)
    all_images = glob(os.path.join(folder, '*.png'))
    selected_images = all_images[:MAX_IMAGES_PER_CLASS] if len(all_images) > MAX_IMAGES_PER_CLASS else all_images
    for img_file in selected_images:
        train_image_paths.append(img_file)
        train_labels.append(class_to_idx[class_name])

# Load Validation Data
val_image_paths, val_labels = [], []
for class_name in SELECTED_CLASSES:
    folder = os.path.join(VALIDATION_PATH, class_name)
    all_images = glob(os.path.join(folder, '*.png'))
    selected_images = all_images[:MAX_IMAGES_PER_CLASS] if len(all_images) > MAX_IMAGES_PER_CLASS else all_images
    for img_file in selected_images:
        val_image_paths.append(img_file)
        val_labels.append(class_to_idx[class_name])


# Data Generator
class AlbumentationsGenerator(Sequence):
    def __init__(self, image_paths, labels, batch_size, transform, num_classes, augmentations_per_image=1):
        self.image_paths = image_paths
        self.labels = labels
        self.batch_size = batch_size
        self.transform = transform
        self.num_classes = num_classes
        self.augmentations_per_image = augmentations_per_image

    def __len__(self):
        return int(np.ceil(len(self.image_paths) * self.augmentations_per_image / self.batch_size))

    def __getitem__(self, idx):
        start = idx * self.batch_size // self.augmentations_per_image
        end = (idx + 1) * self.batch_size // self.augmentations_per_image
        batch_paths = self.image_paths[start:end]
        batch_labels = self.labels[start:end]

        images, labels = [], []
        for img_path, label in zip(batch_paths, batch_labels):
            image = cv2.imread(img_path)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            for _ in range(self.augmentations_per_image):
                augmented = self.transform(image=image)['image']
                images.append(augmented / 255.0)
                labels.append(label)

        return np.array(images, dtype=np.float32), to_categorical(labels, num_classes=self.num_classes)

# Input shape
sample_image = cv2.imread(train_image_paths[0])
sample_image = cv2.cvtColor(sample_image, cv2.COLOR_BGR2RGB)
input_shape = sample_image.shape

# Generators
train_gen = AlbumentationsGenerator(
    train_image_paths, train_labels,
    CONFIG['BATCH_SIZE'], train_transform,
    CONFIG['NUM_CLASSES'], augmentations_per_image=5
)

val_gen = AlbumentationsGenerator(
    val_image_paths, val_labels,
    CONFIG['BATCH_SIZE'], val_transform,
    CONFIG['NUM_CLASSES'], augmentations_per_image=1
)

# Model Architecture
base_model = MobileNetV2(input_shape=input_shape, include_top=False, weights='imagenet')
base_model.trainable = False

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.3)(x)
output = Dense(CONFIG['NUM_CLASSES'], activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=output)
model.compile(
    optimizer=tf.keras.optimizers.Adam(CONFIG['LR']),
    loss=categorical_focal_loss(alpha=0.25, gamma=2.0),
    metrics=['accuracy']
)

# Callbacks
callbacks = [
    EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, verbose=1)
]

# Train the Model
model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=CONFIG['EPOCHS'],
    callbacks=callbacks
)


2025-07-16 12:49:57.971786: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1752670198.158079      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1752670198.214168      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
/tmp/ipykernel_36/79019342.py:49: UserWarning: Argument(s) 'quality_lower, quality_upper' are not valid for transform ImageCompression
  A.ImageCompression(quality_lower=85, quality_upper=100, p=0.2),
/usr/local/lib/python3.11/dist-packages/albumentations/core/validation.py:114: UserWarning: ShiftScaleRotate is a special case of Affine transform. Please use Affine transform instead.
  original_init(self, **validated_kwargs)
I0000 00:

9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/50


I0000 00:00:1752670231.027077      97 service.cc:148] XLA service 0x7cb858002c20 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1752670231.027787      97 service.cc:156]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1752670231.027808      97 service.cc:156]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5
I0000 00:00:1752670232.076422      97 cuda_dnn.cc:529] Loaded cuDNN version 90300


  1/789 ━━━━━━━━━━━━━━━━━━━━ 4:04:23 19s/step - accuracy: 0.0000e+00 - loss: 1.1936

I0000 00:00:1752670240.874366      97 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


789/789 ━━━━━━━━━━━━━━━━━━━━ 515s 630ms/step - accuracy: 0.2041 - loss: 0.6597 - val_accuracy: 0.7511 - val_loss: 0.1837 - learning_rate: 1.0000e-04
Epoch 2/50
789/789 ━━━━━━━━━━━━━━━━━━━━ 386s 489ms/step - accuracy: 0.6819 - loss: 0.1992 - val_accuracy: 0.9164 - val_loss: 0.0591 - learning_rate: 1.0000e-04
Epoch 3/50
789/789 ━━━━━━━━━━━━━━━━━━━━ 407s 516ms/step - accuracy: 0.8215 - loss: 0.1025 - val_accuracy: 0.9526 - val_loss: 0.0332 - learning_rate: 1.0000e-04
Epoch 4/50
789/789 ━━━━━━━━━━━━━━━━━━━━ 391s 496ms/step - accuracy: 0.8796 - loss: 0.0638 - val_accuracy: 0.9674 - val_loss: 0.0206 - learning_rate: 1.0000e-04
Epoch 5/50
789/789 ━━━━━━━━━━━━━━━━━━━━ 401s 508ms/step - accuracy: 0.9069 - loss: 0.0475 - val_accuracy: 0.9687 - val_loss: 0.0183 - learning_rate: 1.0000e-04
Epoch 6/50
789/789 ━━━━━━━━━━━━━━━━━━━━ 360s 456ms/step - accuracy: 0.9250 - loss: 0.0366 - val_accuracy: 0.9736 - val_loss: 0.0134 - learning_rate: 1.0000e-04
Epoch 7/50
789/789 ━━━━━━━━━━━━━━━━━━━━ 389s 493ms/

In [2]:
model.save("asl_227*224.h5")

In [2]:
model.save("asl_asment1.h5")